### Format

    Input
    """
    질문: <question>
    풀이: <reasoning>
    정답: <answer>

    질문: <question>
    풀이: 
    """

    """
    <reasoning>
    정답 : <answer>
    """

In [1]:
import json
import requests
from src import utils
import time
import os
import csv
import re

### Making Training set

In [2]:
# input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [3]:
data = utils.read_jsonlines(input_file)

In [4]:
len(data)

7473

In [5]:
PROMPT1 = "질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? \
풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. \
정답: 72 \\n"


In [6]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [7]:
def get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s):
    prompt = f"질문: {kor_que}"
    output = f"풀이: {kor_ans} 정답: {ans_s}"
    return prompt, output

In [8]:
def get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT1):
    PROMPT1 
    prompt = f"질문: {kor_que} 풀이: "
    output = f"{kor_ans} 정답: {ans_s}"
    
    prompt = PROMPT1 + prompt
    return prompt, output

In [9]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [10]:
# from pprint import pprint

In [11]:
def remove_line_slit(context):
    context = re.sub(r"\n", " ", context)
    return context

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [12]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    
    
    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue
    
#     print(f'kor_que')
#     print(f'{kor_que}')
#     print(f'kor_ans')
#     print(f'{kor_ans}')
#     print(f'ans_s')
#     print(f'{ans_s}')
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT1)
    
    
    input_lst.append(remove_line_slit(in_))
    output_lst.append(remove_line_slit(out_))

#     if cnt == 5:
#         break
#     else:
#         cnt += 1

In [13]:
len(data) - split_cnt

7310

In [14]:
len(input_lst)

7310

In [15]:
len(output_lst)

7310

In [20]:
input_lst[115]

'질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \\n질문: 항아리 안에 몇 개의 젤리빈이 있습니다. 젤리빈의 3/4은 빨간색이며, 빨간색 젤리빈의 1/4은 코코넛 맛입니다. 코코넛 맛 젤리빈이 750개이면, 항아리 안에는 몇 개의 젤리빈이 있나요? 풀이: '

In [22]:
output_lst[115]

'젤리콩이 750*4=3000개가 있습니다. 그리고 병 안에는 3000/3*4=4000개의 젤리콩이 있습니다. 정답: 4000'

In [ ]:
def save_csv_file(file_path, sentence_a_list, sentence_b_list):
    with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        for sentence_a, sentence_b in zip(sentence_a_list, sentence_b_list):
            writer.writerow([sentence_a, sentence_b])

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
output_file = output_path + '/kor-gsm8k-one_shot_cot_finetuning-train.csv'

In [ ]:
output_file

In [ ]:
save_csv_file(output_file, input_lst, output_lst)

In [ ]:
input_lst[1]

In [ ]:
output_lst[1]

### Making Test Set

In [ ]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

In [ ]:
data = utils.read_jsonlines(input_file)

In [ ]:
len(data)

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
output_file = output_path + '/kor-gsm8k-one_shot_cot_finetuning-test.jsonl'

In [ ]:
print(output_file)

In [ ]:
jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
len(data)

In [ ]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    
    
    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT1)
    print('in_')
    print(f'{in_}')
    print('out_')
    print(f'{out_}')
    
    ins['kor_question_reason'] = in_
    ins['kor_question_reason_answer'] = out_

    jsonl_writer.write_json_line(ins)

In [ ]:
len(data) - split_cnt

In [ ]:
split_cnt